In [1]:
import numpy as np
import pandas as pd
import pandas as pd
from textblob import TextBlob
from nltk.tokenize.toktok import ToktokTokenizer
import re
tokenizer = ToktokTokenizer()
import spacy
nlp = spacy.load("en_core_web_sm", disable=["ner"])

In [2]:
TextBlob("he is very good boy").sentiment

Sentiment(polarity=0.9099999999999999, subjectivity=0.7800000000000001)

In [3]:
TextBlob("he is very good boy").sentiment

Sentiment(polarity=0.9099999999999999, subjectivity=0.7800000000000001)

In [4]:
TextBlob("Eerybody says this man is poor").sentiment

Sentiment(polarity=-0.4, subjectivity=0.6)

In [5]:
train=pd.read_csv("/content/Train.csv")
train

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1
...,...,...
39995,"""Western Union"" is something of a forgotten cl...",1
39996,This movie is an incredible piece of work. It ...,1
39997,My wife and I watched this movie because we pl...,0
39998,"When I first watched Flatliners, I was amazed....",1


In [6]:
label_0 = train[train["label"]==0].sample(n=5000)
label_1 = train[train["label"]==1].sample(n=5000)

In [7]:
train = pd.concat([label_1,label_0])
from sklearn.utils import shuffle
train = shuffle(train)

In [8]:
train

,text,label
14129,A humorous voyage into the normally somber fun...,1
24619,Richard Brooks excellent 1967 film of Truman C...,1
6,Finally watched this shocking movie last night...,1
184,Version: Universal / Hong Kong Legends R4 DVD ...,1
13296,Did the movie-makers even preview this before ...,0
...,...,...
33827,I almost saw this at an actual movie theatre (...,0
24050,"When I am watching a film, I am aware that it ...",0
20006,"I'm sorry, but this really does feel like a mo...",0
35636,I like 50s sci-fi movies a lot. I like the rea...,0


In [9]:
train.isnull().sum()

,0
text,0
label,0


In [10]:
import numpy as np
train.replace(r'^\s*$', np.nan, regex=True,inplace=True)
train.dropna(axis = 0, how = "any", inplace = True)

In [11]:
#removing escape seq
train.replace(to_replace = [r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True, inplace=True)


In [12]:
import numpy as np
train.replace(r'^\s*$', np.nan, regex=True,inplace=True)
train.dropna(axis = 0, how = "any", inplace = True)

In [13]:
train

,text,label
14129,A humorous voyage into the normally somber fun...,1
24619,Richard Brooks excellent 1967 film of Truman C...,1
6,Finally watched this shocking movie last night...,1
184,Version: Universal / Hong Kong Legends R4 DVD ...,1
13296,Did the movie-makers even preview this before ...,0
...,...,...
33827,I almost saw this at an actual movie theatre (...,0
24050,"When I am watching a film, I am aware that it ...",0
20006,"I'm sorry, but this really does feel like a mo...",0
35636,I like 50s sci-fi movies a lot. I like the rea...,0


In [14]:
#removing non-ascii data
train["text"] = train["text"].str.encode("ascii", "ignore").str.decode("ascii")

## Removing punctuations

In [15]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [16]:
def remove_punctuations(text):
    import string
    for punctuation in string.punctuation:
        text = text.replace(punctuation, " ")
    return text
train["text"]=train["text"].apply(remove_punctuations)

In [17]:
train

,text,label
14129,A humorous voyage into the normally somber fun...,1
24619,Richard Brooks excellent 1967 film of Truman C...,1
6,Finally watched this shocking movie last night...,1
184,Version Universal Hong Kong Legends R4 DVD ...,1
13296,Did the movie makers even preview this before ...,0
...,...,...
33827,I almost saw this at an actual movie theatre ...,0
24050,When I am watching a film I am aware that it ...,0
20006,I m sorry but this really does feel like a mo...,0
35636,I like 50s sci fi movies a lot I like the rea...,0


## Removing Stopwords

In [19]:
pip install nltk

In [21]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [24]:
#removing "no" and "not" from stopwords
stopword_list = nltk.corpus.stopwords.words("english")
stopword_list.remove("no")
stopword_list.remove("not")

In [25]:
def custom_remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = " ".join(filtered_tokens)
    return filtered_text

In [26]:
#applying stopwords remover function
train['text'] = train['text'].apply(custom_remove_stopwords)

In [27]:
train

,text,label
14129,humorous voyage normally somber funeral busine...,1
24619,Richard Brooks excellent 1967 film Truman Capo...,1
6,Finally watched shocking movie last night dist...,1
184,Version Universal Hong Kong Legends R4 DVD rel...,1
13296,movie makers even preview released script jump...,0
...,...,...
33827,almost saw actual movie theatre art house thea...,0
24050,watching film aware movie nonetheless like all...,0
20006,sorry really feel like modern day Apollo 13 kn...,0
35636,like 50s sci fi movies lot like really good on...,0


## Removing Special Characters

In [28]:
def remove_special_characters(text):
    text = re.sub('[^a-zA-z0-9\s]', " ", text)
    return text

In [29]:
train["text"] = train["text"].apply(remove_special_characters)

In [30]:
train

,text,label
14129,humorous voyage normally somber funeral busine...,1
24619,Richard Brooks excellent 1967 film Truman Capo...,1
6,Finally watched shocking movie last night dist...,1
184,Version Universal Hong Kong Legends R4 DVD rel...,1
13296,movie makers even preview released script jump...,0
...,...,...
33827,almost saw actual movie theatre art house thea...,0
24050,watching film aware movie nonetheless like all...,0
20006,sorry really feel like modern day Apollo 13 kn...,0
35636,like 50s sci fi movies lot like really good on...,0


In [31]:
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r" ",text)

In [33]:
train["text"] = train["text"].apply(remove_URL)
train

,text,label
14129,humorous voyage normally somber funeral busine...,1
24619,Richard Brooks excellent 1967 film Truman Capo...,1
6,Finally watched shocking movie last night dist...,1
184,Version Universal Hong Kong Legends R4 DVD rel...,1
13296,movie makers even preview released script jump...,0
...,...,...
33827,almost saw actual movie theatre art house thea...,0
24050,watching film aware movie nonetheless like all...,0
20006,sorry really feel like modern day Apollo 13 kn...,0
35636,like 50s sci fi movies lot like really good on...,0


In [34]:
def remove_numbers(text):
    """ Removes integers """
    text = "".join([i for i in text if not i.isdigit()])
    return text

In [35]:
train["text"] = train["text"].apply(remove_numbers)
train

,text,label
14129,humorous voyage normally somber funeral busine...,1
24619,Richard Brooks excellent film Truman Capote n...,1
6,Finally watched shocking movie last night dist...,1
184,Version Universal Hong Kong Legends R DVD rele...,1
13296,movie makers even preview released script jump...,0
...,...,...
33827,almost saw actual movie theatre art house thea...,0
24050,watching film aware movie nonetheless like all...,0
20006,sorry really feel like modern day Apollo knoc...,0
35636,like s sci fi movies lot like really good ones...,0


In [36]:
def cleanse(word):
    rx = re.compile(r'\D*\d')
    if rx.match(word):
        return " "
    return word
def remove_alphanumeric(strings):
    nstrings = [" ".join(filter(None, (
    cleanse(word) for word in string.split())))
    for string in strings.split()]
    str1 = " ".join(nstrings)
    return str1

In [37]:
train["text"] = train["text"].apply(remove_alphanumeric)
train


,text,label
14129,humorous voyage normally somber funeral busine...,1
24619,Richard Brooks excellent film Truman Capote no...,1
6,Finally watched shocking movie last night dist...,1
184,Version Universal Hong Kong Legends R DVD rele...,1
13296,movie makers even preview released script jump...,0
...,...,...
33827,almost saw actual movie theatre art house thea...,0
24050,watching film aware movie nonetheless like all...,0
20006,sorry really feel like modern day Apollo knock...,0
35636,like s sci fi movies lot like really good ones...,0


## Lemmatization


In [38]:
def lemmatize_text(text):
    text = nlp(text)
    text = " ".join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

train["text"] = train["text"].apply(lemmatize_text)
train["sentiment"] = train["text"].apply(lambda tweet: TextBlob(tweet).sentiment)
train


,text,label,sentiment
14129,humorous voyage normally somber funeral busine...,1,"(0.09662004662004664, 0.6301864801864802)"
24619,Richard Brooks excellent film Truman Capote no...,1,"(0.18125000000000002, 0.7298611111111111)"
6,finally watch shocking movie last night distur...,1,"(0.06794733044733045, 0.6468614718614718)"
184,version Universal Hong Kong Legends r dvd rele...,1,"(0.16987529515938607, 0.5069165682802046)"
13296,movie maker even preview release script jump p...,0,"(-0.06217948717948718, 0.5685897435897436)"
...,...,...,...
33827,almost see actual movie theatre art house thea...,0,"(0.036421130952380955, 0.5072823660714285)"
24050,watch film aware movie nonetheless like allow ...,0,"(-0.05833333333333334, 0.5166666666666666)"
20006,sorry really feel like modern day Apollo knock...,0,"(-0.09388888888888887, 0.5892592592592593)"
35636,like s sci fi movie lot like really good one D...,0,"(0.20327380952380952, 0.5581547619047619)"


In [39]:
#storing these values into a list
sentiment_series = train["sentiment"].tolist()

In [40]:
columns = ["polarity", "subjectivity"]
df1 = pd.DataFrame(sentiment_series, columns=columns, index=train.index)

In [41]:
df1

,polarity,subjectivity
14129,0.096620,0.630186
24619,0.181250,0.729861
6,0.067947,0.646861
184,0.169875,0.506917
13296,-0.062179,0.568590
...,...,...
33827,0.036421,0.507282
24050,-0.058333,0.516667
20006,-0.093889,0.589259
35636,0.203274,0.558155


In [42]:
result = pd.concat([train, df1], axis=1)

In [43]:
result.drop(["sentiment"], axis=1, inplace=True)

In [44]:
result.loc[result["polarity"]>=0.3, "Sentiment"] = "Positive"
result.loc[result["polarity"]<0.3, "Sentiment"] = "Negative"

In [45]:
result

,text,label,polarity,subjectivity,Sentiment
14129,humorous voyage normally somber funeral busine...,1,0.096620,0.630186,Negative
24619,Richard Brooks excellent film Truman Capote no...,1,0.181250,0.729861,Negative
6,finally watch shocking movie last night distur...,1,0.067947,0.646861,Negative
184,version Universal Hong Kong Legends r dvd rele...,1,0.169875,0.506917,Negative
13296,movie maker even preview release script jump p...,0,-0.062179,0.568590,Negative
...,...,...,...,...,...
33827,almost see actual movie theatre art house thea...,0,0.036421,0.507282,Negative
24050,watch film aware movie nonetheless like allow ...,0,-0.058333,0.516667,Negative
20006,sorry really feel like modern day Apollo knock...,0,-0.093889,0.589259,Negative
35636,like s sci fi movie lot like really good one D...,0,0.203274,0.558155,Negative


In [46]:
result.loc[result['label']==1, 'Sentiment_label'] = 1
result.loc[result['label']==0, 'Sentiment_label'] = 0

In [47]:
result

,text,label,polarity,subjectivity,Sentiment,Sentiment_label
14129,humorous voyage normally somber funeral busine...,1,0.096620,0.630186,Negative,1.0
24619,Richard Brooks excellent film Truman Capote no...,1,0.181250,0.729861,Negative,1.0
6,finally watch shocking movie last night distur...,1,0.067947,0.646861,Negative,1.0
184,version Universal Hong Kong Legends r dvd rele...,1,0.169875,0.506917,Negative,1.0
13296,movie maker even preview release script jump p...,0,-0.062179,0.568590,Negative,0.0
...,...,...,...,...,...,...
33827,almost see actual movie theatre art house thea...,0,0.036421,0.507282,Negative,0.0
24050,watch film aware movie nonetheless like allow ...,0,-0.058333,0.516667,Negative,0.0
20006,sorry really feel like modern day Apollo knock...,0,-0.093889,0.589259,Negative,0.0
35636,like s sci fi movie lot like really good one D...,0,0.203274,0.558155,Negative,0.0
